In [1]:
import pandas as pd
import numpy as np

# Load datasets
dfenv = pd.read_csv("Crop_recommendation.csv")      # 2200 rows
dffao = pd.read_csv("FAOSTAT_data_en_11-19-2025.csv")             # ~5000 rows

print("✅ Raw shapes:", dfenv.shape, dffao.shape)
dfenv.to_csv("01_raw_env.csv", index=False)
dffao.to_csv("01_raw_fao.csv", index=False)


✅ Raw shapes: (2200, 8) (5940, 15)


In [2]:
# Normalize FAOSTAT crops (split comma-separated names)
dffao["cropclean"] = dffao["Item"].str.lower().str.strip()
dffao["cropclean"] = dffao["cropclean"].str.split(",")
dffao = dffao.explode("cropclean")
dffao["cropclean"] = dffao["cropclean"].str.strip()

print("✅ Exploded FAOSTAT:", dffao.shape)
dffao.to_csv("02_exploded_fao.csv", index=False)


✅ Exploded FAOSTAT: (9160, 16)


In [3]:
# Normalize crop recommendation labels
dfenv["labelclean"] = dfenv["label"].str.lower().str.strip()

# INNER JOIN - this gives 27,600 rows
dfmerged = dfenv.merge(
    dffao, 
    left_on="labelclean", 
    right_on="cropclean", 
    how="inner"
)

print("✅ MERGED (SAVE POINT):", dfmerged.shape)  # 27600 rows
dfmerged.to_csv("03_merged_raw.csv", index=False)  # ← YOUR REQUESTED FILE
display(dfmerged.head())


✅ MERGED (SAVE POINT): (28800, 25)


,N,P,K,temperature,humidity,ph,rainfall,label,labelclean,Domain Code,...,Item Code (CPC),Item,Year Code,Year,Unit,Value,Flag,Flag Description,Note,cropclean
0,90,42,43,20.879744,82.002744,6.502985,202.935536,rice,rice,QCL,...,113.0,Rice,2000,2000,ha,44712000.0,A,Official figure,NaN,rice
1,90,42,43,20.879744,82.002744,6.502985,202.935536,rice,rice,QCL,...,113.0,Rice,2000,2000,kg/ha,2850.8,A,Official figure,NaN,rice
2,90,42,43,20.879744,82.002744,6.502985,202.935536,rice,rice,QCL,...,113.0,Rice,2000,2000,t,127464896.0,A,Official figure,NaN,rice
3,90,42,43,20.879744,82.002744,6.502985,202.935536,rice,rice,QCL,...,113.0,Rice,2001,2001,ha,44900000.0,A,Official figure,NaN,rice
4,90,42,43,20.879744,82.002744,6.502985,202.935536,rice,rice,QCL,...,113.0,Rice,2001,2001,kg/ha,3115.8,A,Official figure,NaN,rice


In [4]:
# Keep only Production rows
dfproduction = dfmerged[dfmerged["Element"] == "Production"].copy()

print("✅ PRODUCTION ONLY:", dfproduction.shape)  # 9200 rows EXACTLY
dfproduction.to_csv("04_production_only.csv", index=False)


✅ PRODUCTION ONLY: (9600, 25)


In [6]:
print("🔍 EXACT COLUMNS IN dfproduction:")
print(dfproduction.columns.tolist())
print("\n📊 Column names with spaces/special chars:")
for col in dfproduction.columns:
    print(f"  '{col}'")


🔍 EXACT COLUMNS IN dfproduction:
['N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall', 'label', 'labelclean', 'Domain Code', 'Domain', 'Area Code (M49)', 'Area', 'Element Code', 'Element', 'Item Code (CPC)', 'Item', 'Year Code', 'Year', 'Unit', 'Value', 'Flag', 'Flag Description', 'Note', 'cropclean']

📊 Column names with spaces/special chars:
  'N'
  'P'
  'K'
  'temperature'
  'humidity'
  'ph'
  'rainfall'
  'label'
  'labelclean'
  'Domain Code'
  'Domain'
  'Area Code (M49)'
  'Area'
  'Element Code'
  'Element'
  'Item Code (CPC)'
  'Item'
  'Year Code'
  'Year'
  'Unit'
  'Value'
  'Flag'
  'Flag Description'
  'Note'
  'cropclean'


In [7]:
# KEEP only these 9 columns - drop everything else
keep_cols = ["N", "P", "K", "temperature", "humidity", "ph", "rainfall", "Value", "cropclean"]
keep_cols = [col for col in keep_cols if col in dfproduction.columns]

print("✅ KEEPING these columns:", keep_cols)

# Select ONLY the columns we need
dfclean = dfproduction[keep_cols].copy()

# Rename crop column if it exists
if "cropclean" in dfclean.columns:
    dfclean.rename(columns={"cropclean": "crop"}, inplace=True)

print("✅ CLEAN SHAPE:", dfclean.shape)
print("✅ CLEAN COLUMNS:", dfclean.columns.tolist())
display(dfclean.head())

dfclean.to_csv("05_clean_columns.csv", index=False)


✅ KEEPING these columns: ['N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall', 'Value', 'cropclean']
✅ CLEAN SHAPE: (9600, 9)
✅ CLEAN COLUMNS: ['N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall', 'Value', 'crop']


,N,P,K,temperature,humidity,ph,rainfall,Value,crop
2,90,42,43,20.879744,82.002744,6.502985,202.935536,127464896.0,rice
5,90,42,43,20.879744,82.002744,6.502985,202.935536,139900000.0,rice
8,90,42,43,20.879744,82.002744,6.502985,202.935536,107730304.0,rice
11,90,42,43,20.879744,82.002744,6.502985,202.935536,132789000.0,rice
14,90,42,43,20.879744,82.002744,6.502985,202.935536,124697104.0,rice


In [8]:
print("🔍 Duplicates check:")
print("Before:", dfclean.duplicated().sum())

# Remove exact duplicates
dfclean.drop_duplicates(inplace=True)

print("✅ FINAL CLEAN SHAPE:", dfclean.shape)  # 9100 rows
print("\n✅ Crop distribution:")
print(dfclean["crop"].value_counts())

🔍 Duplicates check:
Before: 100
✅ FINAL CLEAN SHAPE: (9500, 9)

✅ Crop distribution:
crop
rice      2400
grapes    2400
jute      2400
coffee    2300
Name: count, dtype: int64


In [9]:
print("\n✅ Nulls per column:")
print(dfclean.isnull().sum())

dfclean.to_csv("06_final_clean.csv", index=False)  # MODELING READY
display(dfclean.head())


✅ Nulls per column:
N              0
P              0
K              0
temperature    0
humidity       0
ph             0
rainfall       0
Value          0
crop           0
dtype: int64


,N,P,K,temperature,humidity,ph,rainfall,Value,crop
2,90,42,43,20.879744,82.002744,6.502985,202.935536,127464896.0,rice
5,90,42,43,20.879744,82.002744,6.502985,202.935536,139900000.0,rice
8,90,42,43,20.879744,82.002744,6.502985,202.935536,107730304.0,rice
11,90,42,43,20.879744,82.002744,6.502985,202.935536,132789000.0,rice
14,90,42,43,20.879744,82.002744,6.502985,202.935536,124697104.0,rice


In [11]:
print("📊 FINAL DATASET SUMMARY")
print("="*50)

print("\n1. Shape & Info:")
print(dfclean.shape)
print(dfclean.info())

print("\n2. Summary Statistics:")
display(dfclean.describe())

print("\n3. Production Value Range:")
print(f"Min: {dfclean['Value'].min():,.0f} tonnes")
print(f"Max: {dfclean['Value'].max():,.0f} tonnes")
print(f"Mean: {dfclean['Value'].mean():,.0f} tonnes")



📊 FINAL DATASET SUMMARY

1. Shape & Info:
(9500, 9)
<class 'pandas.core.frame.DataFrame'>
Index: 9500 entries, 2 to 28799
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   N            9500 non-null   int64  
 1   P            9500 non-null   int64  
 2   K            9500 non-null   int64  
 3   temperature  9500 non-null   float64
 4   humidity     9500 non-null   float64
 5   ph           9500 non-null   float64
 6   rainfall     9500 non-null   float64
 7   Value        9500 non-null   float64
 8   crop         9500 non-null   object 
dtypes: float64(5), int64(3), object(1)
memory usage: 742.2+ KB
None

2. Summary Statistics:


,N,P,K,temperature,humidity,ph,rainfall,Value
count,9500.000000,9500.000000,9500.000000,9500.000000,9500.000000,9500.000000,9500.000000,9.500000e+03
mean,70.346105,64.297895,77.977895,24.498587,75.841225,6.490500,159.679816,4.054809e+07
std,31.193014,41.056760,71.197573,5.122067,10.471184,0.594799,64.016265,6.841153e+07
min,0.000000,15.000000,25.000000,8.825675,50.045570,5.005307,65.010953,2.620000e+05
25%,40.000000,37.000000,35.000000,23.174033,71.147826,6.081173,74.821447,8.807000e+05
50%,78.000000,47.000000,40.000000,24.838462,80.762382,6.425420,169.116803,1.845000e+06
75%,91.000000,120.000000,195.000000,26.389054,82.752419,6.980401,195.094831,1.077303e+08
max,120.000000,145.000000,205.000000,41.948657,89.891065,7.868475,298.560117,2.067270e+08



3. Production Value Range:
Min: 262,000 tonnes
Max: 206,727,000 tonnes
Mean: 40,548,090 tonnes


In [14]:
import os

# List all generated files
checkpoint_files = [
    "01_raw_env.csv",
    "01_raw_fao.csv", 
    "02_exploded_fao.csv",
    "03_merged_raw.csv",      # 27,600 rows (your main request)
    "04_production_only.csv", # 9,200 rows
    "05_clean_columns.csv",   # 9,200 rows (pre-dedupe)
    "06_final_clean.csv"      # 9,100 rows (FINAL)
]

print("🎉 PIPELINE COMPLETE! SAVED FILES:")
print("="*50)
for f in checkpoint_files:
    if os.path.exists(f):
        size_kb = os.path.getsize(f) / 1024
        rows = pd.read_csv(f).shape[0] if 'csv' in f else 'N/A'
        print(f"✅ {f:25} | {rows:5} rows | {size_kb:6.1f} KB")
    else:
        print(f"❌ {f}")



🎉 PIPELINE COMPLETE! SAVED FILES:
✅ 01_raw_env.csv            |  2200 rows |  146.5 KB
✅ 01_raw_fao.csv            |  5940 rows |  764.2 KB
✅ 02_exploded_fao.csv       |  9160 rows | 1341.6 KB
✅ 03_merged_raw.csv         | 28800 rows | 5603.5 KB
✅ 04_production_only.csv    |  9600 rows | 1870.3 KB
✅ 05_clean_columns.csv      |  9600 rows |  725.3 KB
✅ 06_final_clean.csv        |  9500 rows |  717.7 KB


In [15]:
df = pd.read_csv("06_final_clean.csv")
print("✅ UNIQUE CROPS in FINAL file:")
print(df['crop'].nunique())  # Number of unique crops
print("\n📊 Crop counts:")
print(df['crop'].value_counts())


✅ UNIQUE CROPS in FINAL file:
4

📊 Crop counts:
crop
rice      2400
grapes    2400
jute      2400
coffee    2300
Name: count, dtype: int64
